# Loading packages

In [77]:
%load_ext autoreload

import pandas
import os
import sys
from tqdm import tqdm
import time
from PIL import Image
import numpy

old_std_out = sys.stdout

from google_images_download import google_images_download
gid = google_images_download.googleimagesdownload()   #class instantiation

# Data augmentation
from imgaug import augmenters as iaa

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Loading stats from CSV file

In [13]:
csv_file = "../Datasets/Pokemon/pokemon_alopez247.csv"
with open(csv_file, 'r') as f:
    statistics = pandas.read_csv(f)
# print(statistics.columns.values)

pokemon_names = list(statistics["Name"])
cat_to_label = {cat:i for i,cat in enumerate(pokemon_names)}
label_to_cat = {i:cat for i,cat in enumerate(pokemon_names)}

pokemon_subset = list(statistics[statistics["Generation"]==1]["Name"])
print("Pokemons : "+str(len(pokemon_subset)))
# print(pokemon_subset[113:])

image_output_directory = "../Datasets/Pokemon/Images"

Pokemons : 151


# Scraping images from Google Images

## Images with single pokemon

In [16]:
num_images_per_pokemon = 200
sys.stdout = old_std_out


for pokemon_name in pokemon_subset[140:]:

    print("Scraping for "+pokemon_name)
    directory = image_output_directory+"/"+pokemon_name+"/"
    keywords = "pokemon "+pokemon_name
    
    num_saved = len([f for f in os.listdir(directory) if f.endswith("jpg") or f.endswith("png")])
    offset = num_saved
    limit = offset + (num_images_per_pokemon - num_saved)
    print("\tNumber saved : "+str(num_saved)+", offset = "+str(offset)+", limit = "+str(limit))

    continue_download = True
    if num_saved >= num_images_per_pokemon:
        continue_download = False
    
    while continue_download:
        
        try:
            sys.stdout = open(os.devnull, "w")
            gid.download({
                "keywords":keywords,
                "limit": limit,
                "type": "photo",
                "output_directory":image_output_directory,
                "image_directory": pokemon_name,
                "format": "jpg",
                "offset": offset,
                "chromedriver": "/usr/local/bin/chromedriver",
                "no_numbering": True
            })
            sys.stdout = old_std_out
        except:
            sys.stdout = old_std_out
            print("There was an error with an url, continuing...")
        
        
        num_saved = len([f for f in os.listdir(directory) if f.endswith("jpg") or f.endswith("png")])
        offset = limit
        limit = offset + (num_images_per_pokemon - num_saved)
        print("\tNumber saved : "+str(num_saved)+", offset = "+str(offset)+", limit = "+str(limit))
        
        if num_saved >= num_images_per_pokemon:
            continue_download = False
    
print("All done")

Scraping for Kabutops
	Number saved : 73, offset = 73, limit = 200
	Number saved : 183, offset = 200, limit = 217
	Number saved : 199, offset = 217, limit = 218
	Number saved : 200, offset = 218, limit = 218
Scraping for Aerodactyl
	Number saved : 70, offset = 70, limit = 200
	Number saved : 177, offset = 200, limit = 223
	Number saved : 193, offset = 223, limit = 230
	Number saved : 198, offset = 230, limit = 232
	Number saved : 200, offset = 232, limit = 232
Scraping for Snorlax
	Number saved : 76, offset = 76, limit = 200
	Number saved : 188, offset = 200, limit = 212
	Number saved : 199, offset = 212, limit = 213
	Number saved : 199, offset = 213, limit = 214
	Number saved : 200, offset = 214, limit = 214
Scraping for Articuno
	Number saved : 71, offset = 71, limit = 200
	Number saved : 187, offset = 200, limit = 213
	Number saved : 199, offset = 213, limit = 214
	Number saved : 199, offset = 214, limit = 215
	Number saved : 200, offset = 215, limit = 215
Scraping for Zapdos
	Numbe

## Images with more than one pokemon

In [48]:
output_directory = "../Datasets/Pokemon/MultiLabelImages/"
sys.stdout = old_std_out

sys.stdout = open(os.devnull, "w")
gid.download({
    "keywords": "pokemon season 1 episodes",
    "limit": 120,
    "type": "photo",
    "output_directory":output_directory,
    "no_directory": True,
    "format": "png",
    "offset": 90,
    "chromedriver": "/usr/local/bin/chromedriver",
    "no_numbering": True
})
sys.stdout = old_std_out

print("All done")

All done


# Removing all broken images from folder

In [ ]:
%autoreload 2
import data_processing as DP

root_image_folder = "../Datasets/Pokemon/Images/"
for pokemon_name in pokemon_subset:
    DP.delete_broken_images(root_image_folder+pokemon_name+"/")

# Making all images square

In [38]:
%autoreload 2
import data_processing as DP

root_image_folder = "../Datasets/Pokemon/Images/"
output_folder = "../Datasets/Pokemon/SquareImages/"
for pokemon_name in pokemon_subset[1:]:
    directory = image_output_directory+"/"+pokemon_name+"/"
    output_directory = output_folder+pokemon_name

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        
    print("Input directory : "+str(directory))
    print("Output directory : "+str(output_directory))
    image_files = [i for i in os.listdir(directory) if i.endswith("jpg") or i.endswith("png") or i.endswith("jpeg")]
    time.sleep(0.5)
    
    num_images = len(image_files)
    num_digits = len(str(num_images))
    
    for i,image_path in enumerate(tqdm(image_files, desc="Resizing image : ")):
        new_path = output_folder+pokemon_name+"/"
        new_image = DP.make_square(directory+image_path, max_size=200)
        new_image.save(new_path+str(i).zfill(num_digits)+"_"+pokemon_name+".png", "PNG")



Input directory : ../Datasets/Pokemon/Images/Ivysaur/
Output directory : ../Datasets/Pokemon/SquareImages/Ivysaur


Resizing image : 100%|██████████| 203/203 [00:11<00:00, 18.14it/s]


Input directory : ../Datasets/Pokemon/Images/Venusaur/
Output directory : ../Datasets/Pokemon/SquareImages/Venusaur


Resizing image : 100%|██████████| 209/209 [00:10<00:00, 19.27it/s]


Input directory : ../Datasets/Pokemon/Images/Charmander/
Output directory : ../Datasets/Pokemon/SquareImages/Charmander


...

Resizing image : 100%|██████████| 202/202 [00:17<00:00, 13.48it/s]


Input directory : ../Datasets/Pokemon/Images/Mewtwo/
Output directory : ../Datasets/Pokemon/SquareImages/Mewtwo


Resizing image : 100%|██████████| 203/203 [00:23<00:00, 12.77it/s]


Input directory : ../Datasets/Pokemon/Images/Mew/
Output directory : ../Datasets/Pokemon/SquareImages/Mew


Resizing image : 100%|██████████| 205/205 [00:18<00:00,  7.34it/s]


# Augmenting datasets

In [94]:
%autoreload 2
import data_processing as DP

for pokemon_name in pokemon_subset[12:13]:
    input_folder = "../Datasets/Pokemon/SquareImages/"+pokemon_name+"/"
    print(input_folder)
    output_folder = "../Datasets/Pokemon/AugmentedImages/"+pokemon_name+"/"

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    images = [i for i in os.listdir(input_folder) if i.endswith(".png")]
    DP.data_augmentation(input_folder, output_folder, pokemon_name, images, 200, repetitions=20)
    print("Done.")

../Datasets/Pokemon/SquareImages/Weedle/
Done.
